<a id='top'></a>

# Watershed and Catchment Landcover Metrics

## Aspect and Percent North Covariates
Calculate aspect grid with the DEM used to create the flow network for each processing area (e.g. HUC8 in NHDPlus or
 region for BB, PWS, or Kodiak). We need the DEMs used for each flow direction grid, or we can't calculate covariates at
 the watershed scale.
 * **~~aspect_rch = calculate mean aspect for stream reach (zonal statistics)~~**
 * **~~aspect_cat = calculate mean aspect over catchments (zonal statistics)~~**
 * **north_wtd = create 1/0 grid of north facing cells from aspect grid (north = aspects from 315-45 degrees), ~~~use
  weighted flow accumulation to sum north facing cells, divide by flow accumulation grid to get % of north facing
  cells in each watershed~~~ Use tabulate area to quantify area of North cells (VALUE_1) and non-North (VALUE_0) cells
  for each watershed and calculate percent north as north_wtd = row[VALUE_1]/(row[VALUE_1]+row[VALUE_0])*100 .**
## Elevation Metrics
Calculate elevation metrics for catchment/watershed with temperature data using zonal statistics as table.
* <b> Running "ALL" zonal statistics as it does not allow you to choose two types of statistics
(ex statistics_type = ["MIN","MAX"]) instead set statistics_type="ALL"</b>
### Catchment Elevation Metrics
* **cat_elev_mn = mean elevation for catchment**
* **cat_elev_min = minimum elevation for catchment**
* **cat_elev_max = max elevation for catchment**
* **cat_elev_std = standard deviation of elevation for catchment**
### Watershed Elevation Metrics
* **wtd_elev_mn = mean watershed elevation**
* **wtd_elev_min = min watershed elevation**
* **wtd_elev_max = max watershed elevation**
* **wtd_elev_sd (or cv) = standard deviation of watershed elevation**
## Slope Metrics
Calculate slope metrics for catchment/watershed with temperature data using zonal statistics as table and
statistics_type = "ALL".
### Catchment Slope Metrics
* **cat_slope_mn = mean slope for catchment**
* **cat_slope_min = minimum slope for catchment**
* **cat_slope_max = max slope for catchment**
* **cat_slope_std = standard deviation of slope for catchment**
### Watershed Elevation Metrics
* **wtd_slope_mn = mean watershed slope**
* **wtd_slope_min = min watershed slope**
* **wtd_slope_max = max watershed slope**
* **wtd_slope_sd (or cv) = standard deviation of watershed slope**
## Lake, Wetland and Glacier Cover
Lake/Pond type waterbodies from NHDPLus hydrography for those regions with NHDPlus derived datasets and NHD waterbodies
for those regions with TauDEM derived waterbodies.  Merge all waterbodies together (FTYPE = 390) and use tabulate area
calculate percent cover of lakes in catchments/watersheds.
 * **Created local copies of NHDPlus and NHD waterbodies and exported to T driver here <>  **
Use NLCD grid (wetlands.tif) and tabulate area on wetlands grid with watersheds (sum) / divide by number of cells in
watershed from fac grid, wetlands from NLCD
## Distance from coast
DO NOT WORK ON THIS


## Import modules

In [1]:
import arcpy
import os
import datetime
import time
import sys
arcpy.env.overwriteOutput = True
today = datetime.datetime.now()
# Make the time stamp.
time_stamp = '{:%d%m%Y}'.format(today)
print(time_stamp)

path = os.getcwd()
print (path)
print (sys.base_exec_prefix)

26102021
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\landcover
C:\Users\dwmerrigan\AppData\Local\Programs\ArcGIS\Pro\bin\Python\envs\arcgispro-py3


## Create working output location to store intermediate data
Create folder and gdb to store output data.
Set data_dir to folder containing all AKSSF regional subfolders/geoadatabases
* RS data folder: data_dir = r"W:\GIS\AKSSF"
* DM data folder: data_dir = r"D:\GIS_Temp\AKSSF_BeckyCopy\AKSSF"

In [3]:
# Set AKSSF Data directory
# data_dir = r"W:\GIS\AKSSF"

data_dir = r"D:\GIS_Temp\AKSSF"
arcpy.env.workspace = data_dir
regions = arcpy.ListWorkspaces(workspace_type="Folder")
rois= []
print (regions)

# Path to create output folder/gdb
temppath = r"D:\GIS_temp" # Output folder
dirname = 'AKSSF_land_met'
tempgdbname = 'AKSSF_land_met.gdb'
temp_dir = os.path.join(temppath, dirname)

# Create temporary working gdb
if not arcpy.Exists(temp_dir):
    os.makedirs(temp_dir)
else:
    print('Working Folder already created...', temp_dir)

outcheck = os.path.join(temp_dir, tempgdbname)

if arcpy.Exists(outcheck):
    print ('Output location already exists', outcheck)
    outgdb = outcheck
if not arcpy.Exists(outcheck):
    print('Creating output GDB')
    tempgdb = arcpy.CreateFileGDB_management(temp_dir,tempgdbname)
    print ('Output geodatabase created at', outcheck)
    outgdb = tempgdb.getOutput(0)

['D:\\GIS_temp\\AKSSF\\Bristol_Bay', 'D:\\GIS_temp\\AKSSF\\Cook_Inlet', 'D:\\GIS_temp\\AKSSF\\Copper_River', 'D:\\GIS_temp\\AKSSF\\Kodiak', 'D:\\GIS_temp\\AKSSF\\Prince_William_Sound']
Working Folder already created... D:\GIS_temp\AKSSF_land_met
Output location already exists D:\GIS_temp\AKSSF_land_met\AKSSF_land_met.gdb


In [4]:
# Set regional workspaces from AKSSF data folder and store in list
arcpy.env.workspace = data_dir
# Use All of the cores on the machine
arcpy.env.parallelProcessingFactor = "100%"

regions  = arcpy.ListWorkspaces(workspace_type="Folder")
regions

['D:\\GIS_temp\\AKSSF\\Bristol_Bay',
 'D:\\GIS_temp\\AKSSF\\Cook_Inlet',
 'D:\\GIS_temp\\AKSSF\\Copper_River',
 'D:\\GIS_temp\\AKSSF\\Kodiak',
 'D:\\GIS_temp\\AKSSF\\Prince_William_Sound']

## Create aspect and north grids if they do not already exists
### NHDPlus

In [ ]:
# Code to create merged elevation grid, aspect and north aspect grids for entire AKSSF region with NHDPlus elev data

# elrasters = []
# nhd_dat = r"C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\hydrography\\AKSSF_NHDPlus\\extracts"
# for dirpath, dirname, filenames in arcpy.da.Walk(nhd_dat, 'RasterDataset'):
#     for filename in filenames:
#         if filename == "elev_cm.tif":
#             elras = os.path.join(dirpath, filename)
#             elrasters.append(elras)
#             print (f'Appending {elras} to list')
#
# spatial_ref = arcpy.Describe(elrasters[0]).spatialReference
# print (spatial_ref.name)
# # Merge all nhdplus elevation rasters together
# elevname = 'AKSSF_NHDPlus_elev_cm.tif'
# AKSSF_elev_cm = arcpy.MosaicToNewRaster_management(elrasters,temp_dir,elevname, spatial_ref, "32_BIT_SIGNED", "5", "1", "LAST","FIRST")
#
# #make aspect raster for all of akssf from merged nhdplus
# from arcpy.sa import *
# aspname = 'AKSSF_NHDPlus_aspect.tif'
# asp_rast = Aspect(in_raster= AKSSF_elev_cm, method='Planar')
# asp_rast.save(os.path.join(temp_dir, aspname))
#
# # make north raster from aspect raster
# norname = 'AKSSF_NHDPlus_north_aspect.tif'
# aspect_in = Raster(asp_rast)
# north_raster = Con((aspect_in>=0)&(aspect_in<=45),1,Con((aspect_in<=360)&(aspect_in>=315),1,0))
# north_raster.save(os.path.join(temp_dir, norname))

### TauDEM - Create grids using Timms Composite DEM for entire TauDEM processed areas

In [ ]:
# Code to create merged elevation grid, aspect and north aspect grids for entire AKSSF region with TauDEM elev data
# compDEM = r"C:\Users\dwmerrigan\Documents\GitHub\AKSSF\hydrography\data\topography\AKSSF_Elevation_Composite_10m.tif"
# clip to study area
# AKSSF_elev_10m = r"D:\\GIS_temp\\AKSSF_Composite_10m_extract.tif"
#
# #make aspect raster for all of akssf from merged nhdplus
# from arcpy.sa import *
# aspname = 'AKSSF_Composite_10m_aspect.tif'
# asp_rast = Aspect(in_raster= AKSSF_elev_10m, method='Planar')
# asp_rast.save(os.path.join(temp_dir, aspname))
#
# # make north raster from aspect raster
# norname = 'AKSSF_Composite_10m_north_aspect.tif'
# aspect_in = Raster(asp_rast)
# north_raster = Con((aspect_in>=0)&(aspect_in<=45),1,Con((aspect_in<=360)&(aspect_in>=315),1,0))
# north_raster.save(os.path.join(temp_dir, norname))

## Begin GIS portion
- May be faster and save more space to create aspect rasters here using extract by mask?
- May be able to use aspect raster with tabulate intersection to calculate percent North (aspects from 315-45 degrees)

### <b>UPDATE - 2021-10-12 Do not run aspect metrics at this time. Only need to run percent North for watersheds</b>

In [26]:
from arcpy.sa import *
import re

# Set data_dir equal to folder containing AKSSF regional subfolders containing GDBs and raster datasets
arcpy.env.workspace = data_dir
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338) #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr
regions  = arcpy.ListWorkspaces(workspace_type="Folder")

# Lists for variables not needed at present time
#cat_asp_ztables = []
#wtd_asp_ztables = []
#cat_pernorth_taba_tables=[]

# Lists to store output tables
wtd_pernorth_taba_tables=[]
wtd_lp_tabint_tables = []
wtd_glac_tabint_tables = []
wtd_wet_taba_tables = []
cat_elev_ztables = []
wtd_elev_ztables = []
cat_slope_ztables = []
wtd_slope_ztables = []


# Clear lists
cat_cur_fields = []
wtd_cur_fields = []

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()

# Seperate data by
nhdplus_dat = ['Cook_Inlet','Copper_River']
tauDem_dat = ['Bristol_Bay', 'Kodiak', 'Prince_William_Sound']

# Loop through all processing areas
rois = nhdplus_dat + tauDem_dat
#rois = ['Cook_Inlet']
# Uncomment and change list accordingly below to test individual roi/rois
# rois = ['Kodiak']
for roi in rois:
    # Loop through regional folders
    for region in regions:
        if roi in str(region):
            print(f'{roi} using data from {region} folder')
            # Set data and variables unique to regions with NHDPlus Data
            if roi in nhdplus_dat:
                # # NHDPLus raster data for DM pc, variables will be overwritten below if they exist in source folder.
                # asp_rast = r"D:\Basedata\AKSSF_Basedata\AKSSF_NHDPlus_aspect.tif"
                # elev_rast = r"D:\Basedata\AKSSF_Basedata\AKSSF_NHDPlus_elev_cm.ti
                # f"
                # nor_rast = r"D:\Basedata\AKSSF_Basedata\AKSSF_NHDPlus_north_aspect.tif"
                lakes_fc = r"D:\\Basedata\\AKSSF_Basedata\\AKSSF_Basedata.gdb\\AKSSF_NHDPlus_LakePond_alb"
                # Fields for update cursor
                cat_cur_fields = ['cat_ID_txt', 'NHDPlusID',"cat_ID_con"]
                wtd_cur_fields = ['cat_ID_txt', 'cat_ID',"cat_ID_con"]
                print (f'{roi} in {nhdplus_dat} AKSSF list, using cat_fields {cat_cur_fields} and watershed fields {wtd_cur_fields}')
                # print(f'NHD data: north raster - {nor_rast}')
                # print(f'         aspect raster - {asp_rast}')
                # print(f'      elevation raster - {elev_rast}')
                print('----------')
            # Set data and variables unique to regions with TauDEM Data
            elif roi in tauDem_dat:
                # # TauDEM raster data for DM pc, variables will be overwritten below if they exist in source folder.
                # nor_rast = r"D:\Basedata\AKSSF_Basedata\AKSSF_Composite_10m_north_aspect.tif"
                # asp_rast = r"D:\Basedata\AKSSF_Basedata\AKSSF_Composite_10m_aspect.tif"
                # elev_rast = r"D:\Basedata\AKSSF_Basedata\AKSSF_Composite_10m_extract.tif"

                lakes_fc = r"D:\\Basedata\\AKSSF_Basedata\\AKSSF_Basedata.gdb\\AKSSF_NHD_LakesPonds_alb"
                # Fields for update cursor
                cat_cur_fields = ['cat_ID_txt', 'gridcode',"cat_ID_con"]
                wtd_cur_fields = ['cat_ID_txt', 'cat_ID',"cat_ID_con"]

                print (f'{roi} in {tauDem_dat} TauDEM list, using cat_fields {cat_cur_fields} and watershed fields {wtd_cur_fields}')
                # print(f'TauDem data: north raster - {nor_rast}')
                # print(f'         aspect raster - {asp_rast}')
                # print(f'      elevation raster - {elev_rast}')
                print('----------')

            wtds = []
            cats = []
            # Start iter timing function
            iteration_start = time.time()
            # Set workspace to region folder
            arcpy.env.workspace = region
            gdb = arcpy.ListWorkspaces(workspace_type='FileGDB')
            walk = arcpy.da.Walk(region, datatype = ['FeatureClass','RasterDataset'])
            for dirpath, dirnames, filenames in walk:
                for filename in filenames:
                    # Create list of watersheds
                    if (("wtd" in filename) and ("merge" not in filename)):
                        #print(f" {filename} watershed identified")
                        wtds.append(os.path.join(dirpath, filename))
                    # Set merged watersheds dataset
                    elif 'wtds_merge'in filename:
                        wtdpath = os.path.join(dirpath,filename)
                        wtdname = roi +'_'+filename
                        # Make local copy projected in AKAlbers
                        wtd_merge = arcpy.FeatureClassToFeatureClass_conversion(wtdpath,outgdb,wtdname)
                        print(f'Merged watershed dataset {filename} found')
                        print('----------')
                        wtdfieldnames = []
                        wtdlstFields = arcpy.ListFields(wtd_merge)
                        for field in wtdlstFields:
                            wtdfieldnames.append(field.name)
                        if str(wtd_cur_fields[0]) in wtdfieldnames:
                            print (f'{wtd_cur_fields[0]} field already in dataset')
                            print('----------')
                        else:
                            print (f'Adding {wtd_cur_fields[0]} field to watershed dataset {wtd_merge}')
                            print('----------')
                            # add cat_ID_txt field and concat cat_ID + region
                            arcpy.AddField_management(wtd_merge, str(wtd_cur_fields[0]),field_type='TEXT')
                            # populate cat_ID_txt
                            with arcpy.da.UpdateCursor(wtd_merge, wtd_cur_fields[0:2]) as cur:
                                for row in cur:
                                    int_val = int(row[1])
                                    row[0] = str(int_val)
                                    # Update rows
                                    cur.updateRow(row)
                                del(row)
                            del(cur)
                        if str(wtd_cur_fields[2]) in wtdfieldnames:
                            print (f'{wtd_cur_fields[2]} field already in dataset {wtd_merge}')
                            print('----------')
                        else:
                            print (f'Adding {wtd_cur_fields[2]} field to watershed dataset {wtd_merge}')
                            print('----------')
                            # add cat_ID_con field and concat cat_ID + region
                            arcpy.AddField_management(wtd_merge, str(wtd_cur_fields[2]),field_type='TEXT')
                            # populate cat_ID_txt
                            with arcpy.da.UpdateCursor(wtd_merge, wtd_cur_fields) as cur:
                                for row in cur:
                                    int_val = int(row[1])
                                    row[2] = str(roi) +'_'+ str(int_val)
                                    # Update rows
                                    cur.updateRow(row)
                                del(row)
                            del(cur)

                    # Select glaciers fc
                    elif 'glaciers' in filename:
                        # Make local copy projected in AKAlbers
                        glacpath = os.path.join(dirpath, filename)
                        glacname = roi+'_'+filename
                        glac_fc = arcpy.FeatureClassToFeatureClass_conversion(glacpath,outgdb,glacname)

                    # Select elevation raster
                    elif 'elev.tif' == filename:
                        elev_rast = os.path.join(dirpath, filename)

                    # # Select aspect raster
                    # elif 'aspect' in filename:
                    #     asp_rast = os.path.join(dirpath, filename)

                    # Select north raster
                    elif 'north.tif' == filename:
                        nor_rast = os.path.join(dirpath, filename)

                    # Select slope raster
                    elif 'slope.tif' == filename:
                        slope_rast = os.path.join(dirpath, filename)

                    # Select wetland raster
                    elif 'wetlands.tif' == filename:
                        wet_rast = os.path.join(dirpath, filename)

                    # Select catch_int fc (catchments of interest for region) and make a copy
                    elif 'cats_intersect' in filename:
                        # Make local copy projected in AKAlbers
                        catspath = os.path.join(dirpath,filename)
                        catsname = roi+"_"+filename
                        cats = arcpy.FeatureClassToFeatureClass_conversion(catspath, outgdb,catsname)
                        catlstfields = arcpy.ListFields(cats)
                        catfieldnames = []
                        for field in catlstfields:
                            catfieldnames.append(field.name)
                        if str(cat_cur_fields[0]) in catfieldnames:
                            print (f'{cat_cur_fields[0]} field already in dataset {cats}')
                            print('----------')
                        else:
                            print (f'Adding {cat_cur_fields[0]} field to catchment dataset {cats}')
                            print('----------')
                            # add cat_ID_txt field
                            arcpy.AddField_management(cats, str(cat_cur_fields[0]), field_type='TEXT')
                            # populate cat_ID_txt
                            with arcpy.da.UpdateCursor(cats, cat_cur_fields[0:2]) as cur:
                                for row in cur:
                                    int_val = int(row[1])
                                    row[0] = str(int_val)
                                    # Update rows
                                    cur.updateRow(row)
                                del(row)
                            del(cur)
                        if str(cat_cur_fields[2]) in catfieldnames:
                            print (f'{cat_cur_fields[2]} field already in dataset {cats}')
                            print('----------')
                        else:
                            print (f'Adding {cat_cur_fields[2]} field to catchment dataset {cats}')
                            print('----------')
                            # add cat_ID_txt field & cat_ID + region concat field
                            arcpy.AddField_management(cats,str(cat_cur_fields[2]),field_type='TEXT')
                            # populate cat_ID_con
                            with arcpy.da.UpdateCursor(cats, cat_cur_fields) as cur:
                                for row in cur:
                                    int_val = int(row[1])
                                    row[2] = str(roi) +'_'+ str(int_val)
                                    # Update rows
                                    cur.updateRow(row)
                                del(row)
                            del(cur)

            print (f'Calculating topographic metrics for catchments & watersheds of interest in {roi}'
                   f' region')
            print ('----------')
            print(f'Geodatabase: {gdb}')
            print ('----------')
            print (f'Watershed Merge: {wtd_merge}')
            print (f'  Projection {arcpy.Describe(wtd_merge).spatialReference.name}')
            print ('----------')
            print (f'Catchment Intersect: {cats}')
            print (f'  Projection {arcpy.Describe(cats).spatialReference.name}')
            print ('----------')
            print (f'Elevation Raster: {elev_rast}')
            print (f'  Projection: {arcpy.Describe(elev_rast).spatialReference.name}')
            print ('----------')
            print (f'North Aspect Raster: {nor_rast}')
            print (f'  Projection: {arcpy.Describe(nor_rast).spatialReference.name}')
            print ('----------')
            print (f'Wetlands Raster: {wet_rast}')
            print (f'  Projection {arcpy.Describe(wet_rast).spatialReference.name}')
            print ('----------')
            print (f'Slope Raster: {slope_rast}')
            print (f'  Projection {arcpy.Describe(slope_rast).spatialReference.name}')
            print ('----------')        
            print (f'Lakes Ponds fc: {lakes_fc}')
            print (f'  Projection {arcpy.Describe(lakes_fc).spatialReference.name}')
            print ('----------')
            print (f'Glaciers fc: {glac_fc} ')
            print (f'  Projection {arcpy.Describe(glac_fc).spatialReference.name}')
            print ('----------')
            print (f'{len(wtds)} Watersheds to process')
            print ('----------')
            print (f'Catchment intersect {cats} selected')
            print ('----------')

            try:
                # Set output names/paths for watershed and catchment tables
                wtd_merge_name = roi + "_Watersheds_Merge"
                wtd_merge_path = os.path.join(outgdb,wtd_merge_name)

                # # Aspect variables
                # wtd_merge_asp_table_name = roi + "_Watersheds_Merge_AspectZstats"
                # wtd_merge_asp_table_path = os.path.join(outgdb, wtd_merge_asp_table_name)
                # cat_asp_table_name = roi + "_Catchments_AspectZstats"
                # cat_asp_table_path = os.path.join(outgdb, cat_asp_table_name)

                # Percent North variables
                wtd_merge_pernorth_table_name = roi + "_Watersheds_Merge_PercentNorth"
                wtd_merge_pernorth_table_path = os.path.join(outgdb, wtd_merge_pernorth_table_name)
                # cat_pernorth_table_name = roi + "_Catchments_PercentNorth"
                # cat_pernorth_table_path = os.path.join(outgdb, cat_pernorth_table_name)

                # Elevation variables
                wtd_merge_elev_table_name = roi + "_Watersheds_Merge_ElevZstats"
                wtd_merge_elev_table_path = os.path.join(outgdb, wtd_merge_elev_table_name)
                cat_elev_table_name = roi + "_Catchments_ElevZstats"
                cat_elev_table_path = os.path.join(outgdb, cat_elev_table_name)
                
                # Slope variables
                wtd_merge_slope_table_name = roi + "_Watershed_Merge_SlopeZstats"
                wtd_merge_slope_table_path = os.path.join(outgdb, wtd_merge_slope_table_name)
                cat_slope_table_name = roi + "_Catchments_SlopeZstats"
                cat_slope_table_path = os.path.join(outgdb, cat_slope_table_name)

                # Lakes Ponds variables
                wtd_merge_lp_table_name = roi + "_Watershed_Merge_LakesPonds"
                wtd_merge_lp_table_path = os.path.join(outgdb, wtd_merge_lp_table_name)
                cat_lp_table_name = roi + "_Catchments_LakesPonds"
                cat_lp_path = os.path.join(outgdb, cat_lp_table_name)

                # Wetlands variables
                wtd_merge_wetlands_table_name = roi + "_Watershed_Merge_Wetlands"
                wtd_merge_wetlands_table_path = os.path.join(outgdb, wtd_merge_wetlands_table_name)
                cat_wetlands_table_name = roi + "_Catchments_Wetlands"
                cat_wetlands_table_path = os.path.join(outgdb, cat_wetlands_table_name)

                # Glaciers
                wtd_merge_glac_table_name = roi + "_Watershed_Merge_Glaciers"
                wtd_merge_glac_table_path = os.path.join(outgdb, wtd_merge_glac_table_name)
                cat_glac_table_name = roi + "_Catchments_Glaciers"
                cat_glac_table_path = os.path.join(outgdb, cat_glac_table_name)

                # # Watersheds already merged so no need to run at this time
                # if not arcpy.Exists(wtd_merge):
                #     mergestart = time.time()
                #     # Merge watersheds
                #     wtd_merge = arcpy.Merge_management( wtds, wtd_merge_path ,add_source='ADD_SOURCE_INFO')
                #     # Add wtd_id field
                #     arcpy.AddField_management(wtd_merge,'cat_ID_txt',field_type='TEXT')
                #     arcpy.AddField_management(wtd_merge,"cat_ID_con",field_type='TEXT')
                #     # Add region field
                #     arcpy.AddField_management(wtd_merge,'region',field_type='TEXT')
                #     # Populate watershed id and region information using update cursor - faster than field calc
                #     with arcpy.da.UpdateCursor(wtd_merge,['MERGE_SRC','cat_ID_txt','region','cat_ID_con']) as cur:
                #         for row in cur:
                #             row[1] = re.findall('\d+', row[0])[0]
                #             row[2] = roi
                #             row[3] = + str(roi) + '_'+ str(re.findall('\d+', row[0])[0])
                #             # Update
                #             cur.updateRow(row)
                #         del(row)
                #     del(cur)
                #     mergestop = time.time()
                #     mergetime = int (mergestop - mergestart)
                #     print(f'Watershed Merge for {roi} Elapsed time: ({datetime.timedelta(seconds=mergetime)})')
                #     print('----------')
                # else:
                #     print (f'Merged watershed dataset {wtd_merge} already created')
                #     print('----------')

                # Begin Zonal Stats
                zstat_start = time.time()
                print(f'Begin Elevation zonal statistics min/mean/max std dev for watersheds and catchments in {roi}'
                      f' region')

                # Elevation Zonal statistics  for watersheds
                print(f'Calculating {roi} watershed elevation zonal stats...')
                arcpy.env.snapRaster = elev_rast
                arcpy.env.cellSize = elev_rast
                wtd_elev_metrics_table = ZonalStatisticsAsTable(in_zone_data = wtd_merge,
                                                                zone_field = wtd_cur_fields[0],
                                                                in_value_raster = elev_rast,
                                                                out_table = wtd_merge_elev_table_path,
                                                                statistics_type='ALL'
                                                                )
                # Add region identifier field for watershed tables                                                )
                arcpy.AddField_management(wtd_elev_metrics_table,'region',field_type='TEXT')
                # Add cat_ID_Con field
                arcpy.AddField_management(wtd_elev_metrics_table,'cat_ID_con',field_type='TEXT')
                # Update region field
                with arcpy.da.UpdateCursor(wtd_elev_metrics_table,['region','cat_ID_txt','cat_ID_con']) as cur:
                    for row in cur:
                        row[0] = roi
                        row[2] = roi+"_"+row[1]
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # Append watershed elev table to list
                wtd_elev_ztables.append(wtd_elev_metrics_table)

                # Elevation zonal statistics for catchments
                print(f'Calculating {roi} catchment elevation zonal stats...')
                arcpy.env.snapRaster = elev_rast
                arcpy.env.cellSize = elev_rast
                cat_elev_metrics_table = ZonalStatisticsAsTable(in_zone_data = cats ,
                                                                zone_field = cat_cur_fields[0],
                                                                in_value_raster = elev_rast,
                                                                out_table = cat_elev_table_path,
                                                                statistics_type='ALL'
                                                                )
                # Add region identifier field for catchment table
                arcpy.AddField_management(cat_elev_metrics_table,'region',field_type='TEXT')
                # Add cat_ID_Con field
                arcpy.AddField_management(cat_elev_metrics_table,'cat_ID_con',field_type='TEXT')

                # Update fields
                with arcpy.da.UpdateCursor(cat_elev_metrics_table,['region','cat_ID_txt','cat_ID_con']) as cur:
                    for row in cur:
                        row[0] = roi
                        row[2] = roi+"_"+row[1]
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # Append catchment elev table to list
                cat_elev_ztables.append(cat_elev_metrics_table)

                # Slope zonal statistics for catchments
                print(f'Calculating {roi} catchment slope zonal stats...')
                arcpy.env.snapRaster = slope_rast
                arcpy.env.cellSize = slope_rast
                cat_slope_metrics_table = ZonalStatisticsAsTable(in_zone_data = cats ,
                                                                zone_field = cat_cur_fields[0],
                                                                in_value_raster = slope_rast,
                                                                out_table = cat_slope_table_path,
                                                                statistics_type='ALL'
                                                                )
                # Add region identifier field for catchment table
                arcpy.AddField_management(cat_slope_metrics_table,'region',field_type='TEXT')
                # Add cat_ID_Con field
                arcpy.AddField_management(cat_slope_metrics_table,'cat_ID_con',field_type='TEXT')

                # Update region field
                with arcpy.da.UpdateCursor(cat_slope_metrics_table,['region','cat_ID_txt','cat_ID_con']) as cur:
                    for row in cur:
                        row[0] = roi
                        row[2] = roi+"_"+row[1]
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # Append catchment slope table to list

                cat_slope_ztables.append(cat_slope_metrics_table)
                # Watershed slope Zonal Statistics
                print(f'Begin Slope zonal statistics min/mean/max std dev for watersheds and catchments in {roi}'
                      f' region')

                # Elevation Zonal statistics  for watersheds
                print(f'Calculating {roi} watershed slope zonal stats...')
                arcpy.env.snapRaster = slope_rast
                arcpy.env.cellSize = slope_rast
                wtd_slope_metrics_table = ZonalStatisticsAsTable(in_zone_data = wtd_merge,
                                                                zone_field = wtd_cur_fields[0],
                                                                in_value_raster = slope_rast,
                                                                out_table = wtd_merge_slope_table_path,
                                                                statistics_type='ALL'
                                                                )
                # Add region identifier field for watershed tables                                                )
                arcpy.AddField_management(wtd_slope_metrics_table,'region',field_type='TEXT')
                # Add cat_ID_Con field
                arcpy.AddField_management(wtd_slope_metrics_table,'cat_ID_con',field_type='TEXT')

                # Update region field
                with arcpy.da.UpdateCursor(wtd_slope_metrics_table,['region','cat_ID_txt','cat_ID_con']) as cur:
                    for row in cur:
                        row[0] = roi
                        row[2] = roi+"_"+row[1]
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # Append watershed slope table to list
                wtd_slope_ztables.append(wtd_slope_metrics_table)

                # # Aspect Zonal statistics  for watersheds
                # print(f'Calculating {roi} watershed aspect zonal stats...')
                # wtd_asp_metrics_table = ZonalStatisticsAsTable(in_zone_data = wtd_merge, zone_field ="cat_ID_txt",
                #                                                in_value_raster = asp_rast, out_table = wtd_merge_asp_table_path,
                #                                                statistics_type='ALL')
                # arcpy.AddField_management(wtd_asp_metrics_table, 'region', field_type='TEXT')
                # Add cat_ID_Con field
                # arcpy.AddField_management(wtd_asp_metrics_table,'cat_ID_con',field_type='TEXT')
                # arcpy.CalculateField_management(wtd_asp_metrics_table, 'region', 'roi')
                # Update region field
                # with arcpy.da.UpdateCursor(wtd_asp_metrics_table,['region','cat_ID_txt','cat_ID_con']) as cur:
                #     for row in cur:
                #         row[0] = roi
                #         row[2] = roi+"_"+row[1]
                #         # Update
                #         cur.updateRow(row)
                #     del(row)
                # del(cur)
                # wtd_asp_ztables.append(wtd_asp_metrics_table)

                # # Aspect Zonal statistics for catchments
                # print(f'Calculating {roi} catchment aspect zonal stats...')
                # cat_asp_metrics_table = ZonalStatisticsAsTable(in_zone_data = cats, zone_field ="cat_ID_txt",
                #                                                in_value_raster = asp_rast, out_table = cat_asp_table_path,
                #                                                statistics_type='ALL')
                # arcpy.AddField_management(cat_asp_metrics_table, 'region', field_type='TEXT')
                # Add cat_ID_Con field
                # arcpy.AddField_management(cat_asp_metrics_table,'cat_ID_con',field_type='TEXT')
                # arcpy.CalculateField_management(cat_asp_metrics_table, 'region', 'roi')
                # Update region field
                # with arcpy.da.UpdateCursor(cat_asp_metrics_table,['region','cat_ID_txt','cat_ID_con']) as cur:
                #     for row in cur:
                #         row[0] = roi
                #         row[2] = roi+"_"+row[1]
                #         # Update
                #         cur.updateRow(row)
                #     del(row)
                # del(cur)
                # cat_asp_ztables.append(cat_asp_metrics_table)

                zstat_stop = time.time()
                zstat_time = int (zstat_stop - zstat_start)
                print(f'Zonal Stats for {roi} Elapsed time: ({datetime.timedelta(seconds=zstat_time)})')
                print('----------')

                # Tabulate Area with north grid and watersheds
                tabarea_start = time.time()
                print(f'Begin tabulate area of north facing cells for watersheds and catchments in {roi} region')
                print('----------')
                # Percent North Tabulate area for watersheds
                wtd_per_north_tabarea = arcpy.sa.TabulateArea(in_zone_data= wtd_merge,
                                                              zone_field= wtd_cur_fields[0],
                                                              in_class_data=nor_rast,
                                                              class_field="Value",
                                                              out_table=wtd_merge_pernorth_table_path
                                                              )
                # Add region and percent north fields
                arcpy.AlterField_management(wtd_per_north_tabarea,'CAT_ID_TXT','CAT_ID_TXT_DEL','CAT_ID_TXT_DEL')
                arcpy.AddField_management(wtd_per_north_tabarea, 'region', field_type='TEXT')
                arcpy.AddField_management(wtd_per_north_tabarea, 'wtd_north_per', field_type='Float')
                arcpy.AddField_management(wtd_per_north_tabarea, wtd_cur_fields[0], field_type='TEXT')
                arcpy.AddField_management(wtd_per_north_tabarea, wtd_cur_fields[2], field_type='TEXT')
                wtdnorfields = [f.name for f in arcpy.ListFields(wtd_per_north_tabarea)]
                print (wtdnorfields)
                with arcpy.da.UpdateCursor(wtd_per_north_tabarea, wtdnorfields) as cur:
                    for row in cur:
                        row[4] = roi
                        row[5] = row[3]/(row[3]+row[2])*100
                        row[6] = row[1]
                        row[7] = roi +'_'+ row[1]
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # Drop UPPERCASE field form tab area
                arcpy.DeleteField_management(wtd_per_north_tabarea,'CAT_ID_TXT_DEL')
                # Append watershed percent north table to list
                wtd_pernorth_taba_tables.append(wtd_per_north_tabarea)

                # Percent Lakes Ponds using Tabulate Intersection for watersheds
                print(f'Begin tabulate intersection between {lakes_fc} and watersheds in {roi} region')
                print('----------')
                wtd_lp_tabint = arcpy.TabulateIntersection_analysis(wtd_merge,
                                                                    zone_fields=wtd_cur_fields[0],
                                                                    in_class_features=lakes_fc,
                                                                    out_table=wtd_merge_lp_table_path,
                                                                    class_fields='Ftype',
                                                                    out_units="SQUARE_METERS"
                                                                    )
                # Add region and cat id fields
                arcpy.AlterField_management(wtd_lp_tabint,'PERCENTAGE','wtd_lake_per','wtd_lake_per')
                arcpy.AlterField_management(wtd_lp_tabint,'AREA','wtd_lake_area_sqm','wtd_lake_area_sqm')
                arcpy.AddField_management(wtd_lp_tabint, 'region', field_type='TEXT')
                arcpy.AddField_management(wtd_lp_tabint, wtd_cur_fields[1], field_type='TEXT')
                arcpy.AddField_management(wtd_lp_tabint, wtd_cur_fields[2], field_type='TEXT')
                wtdlpfields = [f.name for f in arcpy.ListFields(wtd_lp_tabint)]
                print (wtdlpfields)
                with arcpy.da.UpdateCursor(wtd_lp_tabint, wtdlpfields) as cur:
                    for row in cur:
                        row[5] = roi
                        row[6] = row[1]
                        row[7] = roi +'_'+ row[1]
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)

                # Append watershed lakes ponds table to list
                wtd_lp_tabint_tables.append(wtd_lp_tabint)

                # Percent glaciers using Tabulate Intersection for watersheds
                print(f'Begin tabulate intersection between {glac_fc} and watersheds in {roi} region')
                print('----------')
                wtd_glac_tabint = arcpy.TabulateIntersection_analysis(wtd_merge,
                                                                    zone_fields=wtd_cur_fields[0],
                                                                    in_class_features=glac_fc,
                                                                    out_table=wtd_merge_glac_table_path,
                                                                    class_fields='O1Region',
                                                                    out_units="SQUARE_METERS"
                                                                    )
                # Add region and cat id fields
                arcpy.AlterField_management(wtd_glac_tabint,'PERCENTAGE','wtd_glacier_per','wtd_glacier_per')
                arcpy.AlterField_management(wtd_glac_tabint,'AREA','wtd_glacier_area_sqm','wtd_glacier_area_sqm')
                arcpy.AddField_management(wtd_glac_tabint, 'region', field_type='TEXT')
                arcpy.AddField_management(wtd_glac_tabint, wtd_cur_fields[1], field_type='TEXT')
                arcpy.AddField_management(wtd_glac_tabint, wtd_cur_fields[2], field_type='TEXT')
                wtdglacfields = [f.name for f in arcpy.ListFields(wtd_glac_tabint)]
                print (wtdglacfields)
                with arcpy.da.UpdateCursor(wtd_glac_tabint, wtdglacfields) as cur:
                    for row in cur:
                        row[5] = roi
                        row[6] = row[1]
                        row[7] = roi +'_'+ row[1]
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # Append watershed percent north table to list
                wtd_glac_tabint_tables.append(wtd_glac_tabint)

                # Tabulate Area with wetlands grid and watersheds
                print(f'Begin tabulate intersection between {wet_rast} and watersheds in {roi} region')
                print('----------')
                # Percent North Tabulate area for watersheds
                wtd_per_wet_tabarea = arcpy.sa.TabulateArea(in_zone_data= wtd_merge,
                                                              zone_field= wtd_cur_fields[0],
                                                              in_class_data=wet_rast,
                                                              class_field="Value",
                                                              out_table=wtd_merge_wetlands_table_path
                                                              )
                # Add region and percent wet fields
                arcpy.AlterField_management(wtd_per_wet_tabarea,'CAT_ID_TXT','CAT_ID_TXT_DEL','CAT_ID_TXT_DEL')
                arcpy.AddField_management(wtd_per_wet_tabarea, 'region', field_type='TEXT')
                arcpy.AddField_management(wtd_per_wet_tabarea, 'wtd_wet_per', field_type='Float')
                arcpy.AddField_management(wtd_per_wet_tabarea, wtd_cur_fields[0], field_type='TEXT')
                arcpy.AddField_management(wtd_per_wet_tabarea, wtd_cur_fields[2], field_type='TEXT')
                wtdwetfields = [f.name for f in arcpy.ListFields(wtd_per_wet_tabarea)]
                print (wtdwetfields)
                with arcpy.da.UpdateCursor(wtd_per_wet_tabarea, wtdwetfields) as cur:
                    for row in cur:
                        row[4] = roi
                        row[5] = row[3]/(row[3]+row[2])*100
                        row[6] = row[1]
                        row[7] = roi +'_'+ row[1]
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # Drop UPPERCASE field form tab area
                arcpy.DeleteField_management(wtd_per_wet_tabarea,'CAT_ID_TXT_DEL')
                # Append watershed percent north table to list
                wtd_wet_taba_tables.append(wtd_per_wet_tabarea)

                # # Percent North Tabulate Area for catchments
                # cat_per_north_tabarea = arcpy.sa.TabulateArea(in_zone_data= cats, zone_field='cat_ID_con',
                #                                             in_class_data=nor_rast,"Value",
                #                                             out_table=cat_pernorth_table_path)

                # # Add and calculate region identifier field for catchment table
                # arcpy.AlterField_management(cat_per_north_tabarea,'CAT_ID_TXT','CAT_ID_TXT_DEL','CAT_ID_TXT_DEL')
                # arcpy.AddField_management(cat_per_north_tabarea, 'region', field_type='TEXT')
                # arcpy.AddField_management(cat_per_north_tabarea, 'cat_north_per', field_type='Float')
                # arcpy.AddField_management(cat_per_north_tabarea, cat_cur_fields[0], field_type='TEXT')
                # arcpy.AddField_management(cat_per_north_tabarea, cat_cur_fields[2], field_type='TEXT')
                # catnorfields = [f.name for f in arcpy.ListFields(cat_per_north_tabarea)]
                # print (catnorfields)
                # with arcpy.da.UpdateCursor(cat_per_north_tabarea,catnorfields) as cur:
                #     for row in cur:
                #         row[4] = roi
                #         row[5] = row[3]/(row[3]+row[2])*100
                #         row[6] = row[1]
                #         row[7] = roi +'_'+ row[1]
                #         # Update
                #         cur.updateRow(row)
                #     del(row)
                # del(cur)
                # Drop UPPERCASE field form tab area
                # arcpy.DeleteField_management(cat_per_north_tabarea,'CAT_ID_TXT_DEL')
                # # Append catchment percent north table to list
                # cat_pernorth_taba_tables.append(cat_per_north_tabarea)

                tabarea_stop = time.time()
                tabarea_time = int (tabarea_stop - tabarea_start)
                print(f'Tabulate area/intersections for {roi} Elapsed time: ({datetime.timedelta(seconds=tabarea_time)})')
                print('----------')

            except:
                e = sys.exc_info()[1]
                print(e.args[0])
                arcpy.AddError(e.args[0])

            iter_stop = time.time()
            iter_time = int(iter_stop - iteration_start)
            print(f'All Covariates for {roi} completed. Elapsed time: ({datetime.timedelta(seconds=iter_time)})')
            print('----------')

    else:
        print(f'Region {str(roi)} not found in {region}')

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print('----------')



Cook_Inlet using data from D:\GIS_temp\AKSSF\Cook_Inlet folder
Cook_Inlet in ['Cook_Inlet', 'Copper_River'] AKSSF list, using cat_fields ['cat_ID_txt', 'NHDPlusID', 'cat_ID_con'] and watershed fields ['cat_ID_txt', 'cat_ID', 'cat_ID_con']
----------
cat_ID_txt field already in dataset D:\GIS_temp\AKSSF_land_met\AKSSF_land_met.gdb\Cook_Inlet_cats_intersect
----------
cat_ID_con field already in dataset D:\GIS_temp\AKSSF_land_met\AKSSF_land_met.gdb\Cook_Inlet_cats_intersect
----------
Merged watershed dataset wtds_merge found
----------
cat_ID_txt field already in dataset
----------
cat_ID_con field already in dataset D:\GIS_temp\AKSSF_land_met\AKSSF_land_met.gdb\Cook_Inlet_wtds_merge
----------
Calculating topographic metrics for catchments & watersheds of interest in Cook_Inlet region
----------
Geodatabase: ['D:\\GIS_temp\\AKSSF\\Cook_Inlet\\Cook_Inlet.gdb']
----------
Watershed Merge: D:\GIS_temp\AKSSF_land_met\AKSSF_land_met.gdb\Cook_Inlet_wtds_merge
  Projection NAD_1983_Alaska_Alb

## Drop unnecessary fields and rename as needed from merged tables.
- Create Key value dictionary and use update cursor to rename fields.

In [28]:
# Lists to store output tables
# wtd_pernorth_taba_tables=[]
# wtd_lp_tabint_tables = []
# wtd_glac_tabint_tables = []
# wtd_wet_taba_tables = []
# cat_elev_ztables = []
# wtd_elev_ztables = []
# cat_slope_ztables = []
# wtd_slope_ztables = []

# Table names/paths
wtd_per_north_table_out = os.path.join(outgdb, 'AKSSF_wtd_north_per')
cat_elev_table_out = os.path.join(outgdb,'AKSSF_cat_elev')
cat_slope_table_out = os.path.join(outgdb,'AKSSF_cat_slope')
wtd_elev_table_out = os.path.join(outgdb, 'AKSSF_wtd_elev')
wtd_per_glac_table_out = os.path.join(outgdb, 'AKSSF_wtd_glacier_per')
wtd_per_lp_table_out = os.path.join(outgdb, 'AKSSF_wtd_lakepond_per')
wtd_slope_table_out = os.path.join(outgdb, 'AKSSF_wtd_slope')
wtd_wet_table_out = os.path.join(outgdb, 'AKSSF_wtd_wetland_per')

# Merge all regional tables together
outtables = []
wtd_per_north = arcpy.Merge_management(wtd_pernorth_taba_tables, wtd_per_north_table_out)
arcpy.AlterField_management(wtd_per_north,"VALUE_0","non_north_area","non_north_area")
arcpy.AlterField_management(wtd_per_north,"VALUE_1","north_area","north_area")
outtables.append(wtd_per_north)
cat_elev = arcpy.Merge_management(cat_elev_ztables, cat_elev_table_out)
outtables.append(cat_elev)
wtd_elev = arcpy.Merge_management(wtd_elev_ztables, wtd_elev_table_out)
outtables.append(wtd_elev)
wtd_slope = arcpy.Merge_management(wtd_slope_ztables, wtd_slope_table_out)
outtables.append(wtd_slope)
cat_slope = arcpy.Merge_management(cat_slope_ztables, cat_slope_table_out)
outtables.append(cat_slope)
wtd_wet = arcpy.Merge_management(wtd_wet_taba_tables, wtd_wet_table_out)
arcpy.AlterField_management(wtd_wet,"VALUE_0","non_wetland_area","non_wetland_area")
arcpy.AlterField_management(wtd_wet,"VALUE_1","wetland_area","wetland_area")
outtables.append(wtd_wet)
wtd_glac = arcpy.Merge_management(wtd_glac_tabint_tables, wtd_per_glac_table_out)
outtables.append(wtd_glac)
wtd_lp = arcpy.Merge_management(wtd_lp_tabint_tables, wtd_per_lp_table_out)
outtables.append(wtd_lp)
print ('Tables merged')
print('----------')

Tables merged
----------


In [29]:
#Set up field dictionary
elevDict = { 'ZONE_CODE': ('cat_elev_ZONE_CODE', 'wtd_elev_ZONE_CODE'),
         'COUNT': ('cat_elev_COUNT', 'wtd_elev_COUNT'),
          'AREA': ('cat_elev_AREA', 'wtd_elev_AREA'),
          'MIN': ('cat_elev_MIN', 'wtd_elev_MIN'),
          'MAX': ('cat_elev_MAX', 'wtd_elev_MAX'),
          'RANGE': ('cat_elev_RANGE', 'wtd_elev_RANGE'),
          'MEAN': ('cat_elev_MEAN', 'wtd_elev_MEAN'),
          'STD': ('cat_elev_STD', 'wtd_elev_STD'),
          'SUM': ('cat_elev_SUM', 'wtd_elev_SUM'),
          'VARIETY': ('cat_elev_VARIETY', 'wtd_elev_VARIETY'),
          'MAJORITY': ('cat_elev_MAJORITY', 'wtd_elev_MAJORITY'),
          'MINORITY': ('cat_elev_MINORITY', 'wtd_elev_MINORITY'),
          'MEDIAN': ('cat_elev_MEDIAN', 'wtd_elev_MEDIAN'),
          'PCT90': ('cat_elev_PCT90', 'wtd_elev_PCT90')
         }

slopeDict = { 'ZONE_CODE': ('cat_slope_ZONE_CODE', 'wtd_slope_ZONE_CODE'),
         'COUNT': ('cat_slope_COUNT', 'wtd_slope_COUNT'),
          'AREA': ('cat_slope_AREA', 'wtd_slope_AREA'),
          'MIN': ('cat_slope_MIN', 'wtd_slope_MIN'),
          'MAX': ('cat_slope_MAX', 'wtd_slope_MAX'),
          'RANGE': ('cat_slope_RANGE', 'wtd_slope_RANGE'),
          'MEAN': ('cat_slope_MEAN', 'wtd_slope_MEAN'),
          'STD': ('cat_slope_STD', 'wtd_slope_STD'),
          'SUM': ('cat_slope_SUM', 'wtd_slope_SUM'),
          'VARIETY': ('cat_slope_VARIETY', 'wtd_slope_VARIETY'),
          'MAJORITY': ('cat_slope_MAJORITY', 'wtd_slope_MAJORITY'),
          'MINORITY': ('cat_slope_MINORITY', 'wtd_slope_MINORITY'),
          'MEDIAN': ('cat_slope_MEDIAN', 'wtd_slope_MEDIAN'),
          'PCT90': ('cat_slope_PCT90', 'wtd_slope_PCT90')
         }

# Rename fields for elevation tables
for field in arcpy.ListFields(wtd_elev):
    keyval = field.name
    if keyval in elevDict:
        newname = elevDict[keyval][1]
        newalias = elevDict[keyval][1]
        print (keyval, newname)
        arcpy.AlterField_management(wtd_elev, keyval, newname, newalias)

for field in arcpy.ListFields(cat_elev):
    keyval = field.name
    if keyval in elevDict:
        newname = elevDict[keyval][0]
        newalias = elevDict[keyval][0]
        print (keyval, newname)
        arcpy.AlterField_management(cat_elev, keyval, newname, newalias)

# Rename fields for slope tables
for field in arcpy.ListFields(wtd_slope):
    keyval = field.name
    if keyval in slopeDict:
        newname = slopeDict[keyval][1]
        newalias = slopeDict[keyval][1]
        print (keyval, newname)
        arcpy.AlterField_management(wtd_slope, keyval, newname, newalias)

for field in arcpy.ListFields(cat_slope):
    keyval = field.name
    if keyval in slopeDict:
        newname = slopeDict[keyval][0]
        newalias = slopeDict[keyval][0]
        print (keyval, newname)
        arcpy.AlterField_management(cat_slope, keyval, newname, newalias)

ZONE_CODE wtd_elev_ZONE_CODE
COUNT wtd_elev_COUNT
AREA wtd_elev_AREA
MIN wtd_elev_MIN
MAX wtd_elev_MAX
RANGE wtd_elev_RANGE
MEAN wtd_elev_MEAN
STD wtd_elev_STD
SUM wtd_elev_SUM
VARIETY wtd_elev_VARIETY
MAJORITY wtd_elev_MAJORITY
MINORITY wtd_elev_MINORITY
MEDIAN wtd_elev_MEDIAN
PCT90 wtd_elev_PCT90
ZONE_CODE cat_elev_ZONE_CODE
COUNT cat_elev_COUNT
AREA cat_elev_AREA
MIN cat_elev_MIN
MAX cat_elev_MAX
RANGE cat_elev_RANGE
MEAN cat_elev_MEAN
STD cat_elev_STD
SUM cat_elev_SUM
VARIETY cat_elev_VARIETY
MAJORITY cat_elev_MAJORITY
MINORITY cat_elev_MINORITY
MEDIAN cat_elev_MEDIAN
PCT90 cat_elev_PCT90
ZONE_CODE wtd_slope_ZONE_CODE
COUNT wtd_slope_COUNT
AREA wtd_slope_AREA
MIN wtd_slope_MIN
MAX wtd_slope_MAX
RANGE wtd_slope_RANGE
MEAN wtd_slope_MEAN
STD wtd_slope_STD
SUM wtd_slope_SUM
MEDIAN wtd_slope_MEDIAN
PCT90 wtd_slope_PCT90
ZONE_CODE cat_slope_ZONE_CODE
COUNT cat_slope_COUNT
AREA cat_slope_AREA
MIN cat_slope_MIN
MAX cat_slope_MAX
RANGE cat_slope_RANGE
MEAN cat_slope_MEAN
STD cat_slope_STD


In [17]:
for table in outtables:
    print (arcpy.Describe(table).basename)

AKSSF_wtd_north_per
AKSSF_cat_elev
AKSSF_wtd_elev
AKSSF_wtd_slope
AKSSF_cat_slope
AKSSF_wtd_wetland_per
AKSSF_wtd_glacier_per
AKSSF_wtd_lakepond_per


In [34]:
# Export copies of dbf tables as csv
outdir = r"C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers"
for table in outtables:
    tablename = arcpy.Describe(table).basename + ".csv"
    tablepath = os.path.join(outdir,tablename)
    print( tablepath)
    arcpy.conversion.TableToTable(table, outdir, tablename)

C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\AKSSF_wtd_north_per.csv
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\AKSSF_cat_elev.csv
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\AKSSF_wtd_elev.csv
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\AKSSF_wtd_slope.csv
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\AKSSF_cat_slope.csv
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\AKSSF_wtd_wetland_per.csv
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\AKSSF_wtd_glacier_per.csv
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\AKSSF_wtd_lakepond_per.csv


In [51]:
# Merge all
alltables = os.path.join(outgdb, "AKSSF_Covariates")
alltablesname = 'AKSSF_Covariates.csv'
akssf_cov_tables_merge = arcpy.Merge_management(outtables, alltables)
arcpy.conversion.TableToTable(akssf_cov_tables_merge, outdir, alltablesname)

<Result 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\data_preparation\\sensitivity_drivers\\AKSSF_Covariates.csv'>

## Convert tables to pandas df and explore results
* **Merge tables and export to csv**

In [35]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format # only display 2 decimal places
# list to store covariate data frames
dfs = []

In [36]:
# Make catchment elev df
cat_df = pd.DataFrame()
cat_field_list = []
for field in arcpy.ListFields(cat_elev):
    cat_field_list.append(field.name)
cat_elev_arr = arcpy.da.TableToNumPyArray(cat_elev,cat_field_list)
cat_df = pd.DataFrame(cat_elev_arr)
cat_df = cat_df.drop(["OBJECTID","cat_elev_ZONE_CODE"],axis=1)
cat_df = cat_df.set_index('cat_ID_txt')
dfs.append(cat_df)
cat_df

,cat_elev_COUNT,cat_elev_AREA,cat_elev_MIN,cat_elev_MAX,cat_elev_RANGE,cat_elev_MEAN,cat_elev_STD,cat_elev_SUM,cat_elev_VARIETY,cat_elev_MAJORITY,cat_elev_MINORITY,cat_elev_MEDIAN
cat_ID_txt,,,,,,,,,,,,
75004200007057.0,38470.00,961750.00,0,908,908,346.00,251.07,13310762.00,909,4,908,333
75004300006312.0,57563.00,1439075.00,59,98,39,76.31,9.37,4392462.00,40,69,59,74
75004300001906.0,12127.00,303175.00,3,146,143,27.85,21.37,337746.00,142,9,132,21
75004300000100.0,78759.00,1968975.00,33,78,45,58.67,8.23,4620998.00,46,65,33,59
75004300004983.0,113568.00,2839200.00,21,479,458,265.10,100.44,30107239.00,459,336,24,282
...,...,...,...,...,...,...,...,...,...,...,...,...
43933,3776.00,377600.00,0,512,512,222.92,164.12,841739.00,512,4,32,222
43973,2363.00,236300.00,1,362,361,52.16,67.87,123252.00,258,4,141,20
44553,3888.00,388800.00,0,35,35,4.44,3.49,17249.00,34,5,17,4


In [37]:
# Make catchment slope df
cat_sl_df = pd.DataFrame()
cat_sl_field_list = []
for field in arcpy.ListFields(cat_slope):
    cat_sl_field_list.append(field.name)
    print (field.name, field.aliasName)
cat_sl_arr = arcpy.da.TableToNumPyArray(cat_slope, cat_sl_field_list)
cat_sl_df = pd.DataFrame(cat_sl_arr)
cat_sl_df = cat_sl_df.drop(["OBJECTID", "cat_slope_ZONE_CODE"],axis=1)
cat_sl_df = cat_sl_df.set_index('cat_ID_txt')
dfs.append(cat_sl_df)
cat_sl_df

OBJECTID OBJECTID
cat_ID_txt cat_ID_txt
cat_slope_ZONE_CODE cat_slope_ZONE_CODE
cat_slope_COUNT cat_slope_COUNT
cat_slope_AREA cat_slope_AREA
cat_slope_MIN cat_slope_MIN
cat_slope_MAX cat_slope_MAX
cat_slope_RANGE cat_slope_RANGE
cat_slope_MEAN cat_slope_MEAN
cat_slope_STD cat_slope_STD
cat_slope_SUM cat_slope_SUM
cat_slope_MEDIAN cat_slope_MEDIAN
cat_slope_PCT90 cat_slope_PCT90
region region
cat_ID_con cat_ID_con


,cat_slope_COUNT,cat_slope_AREA,cat_slope_MIN,cat_slope_MAX,cat_slope_RANGE,cat_slope_MEAN,cat_slope_STD,cat_slope_SUM,cat_slope_MEDIAN,cat_slope_PCT90,region,cat_ID_con
cat_ID_txt,,,,,,,,,,,,
75004200007057.0,38470.00,961750.00,0.00,51.53,51.53,24.00,11.34,923424.18,26.77,36.20,Cook_Inlet,Cook_Inlet_75004200007057.0
75004300006312.0,57563.00,1439075.00,0.00,25.87,25.87,3.99,3.30,229592.37,3.91,8.12,Cook_Inlet,Cook_Inlet_75004300006312.0
75004300001906.0,12127.00,303175.00,0.00,42.81,42.81,9.36,8.12,113559.70,6.35,22.22,Cook_Inlet,Cook_Inlet_75004300001906.0
75004300000100.0,78759.00,1968975.00,0.00,51.12,51.12,4.12,4.13,324142.71,3.79,8.95,Cook_Inlet,Cook_Inlet_75004300000100.0
75004300004983.0,113568.00,2839200.00,0.00,58.81,58.81,17.92,10.56,2035012.13,16.38,32.59,Cook_Inlet,Cook_Inlet_75004300004983.0
...,...,...,...,...,...,...,...,...,...,...,...,...
43933,3776.00,377600.00,0.00,1.05,1.05,0.35,0.19,1323.08,0.39,0.56,Prince_William_Sound,Prince_William_Sound_43933
43973,2363.00,236300.00,0.00,0.89,0.89,0.16,0.16,372.47,0.11,0.34,Prince_William_Sound,Prince_William_Sound_43973
44553,3888.00,388800.00,0.00,0.23,0.23,0.04,0.04,161.89,0.03,0.09,Prince_William_Sound,Prince_William_Sound_44553


In [38]:
# Make watershed elev df
wtd_df = pd.DataFrame()
wtd_field_list = []
for field in arcpy.ListFields(wtd_elev):
    wtd_field_list.append(field.name)

wtd_elev_arr = arcpy.da.TableToNumPyArray(wtd_elev,wtd_field_list)
wtd_df = pd.DataFrame(wtd_elev_arr)
wtd_df = wtd_df.drop(["OBJECTID","wtd_elev_ZONE_CODE"],axis=1)
wtd_df = wtd_df.set_index('cat_ID_txt')
dfs.append(wtd_df)
wtd_df


,wtd_elev_COUNT,wtd_elev_AREA,wtd_elev_MIN,wtd_elev_MAX,wtd_elev_RANGE,wtd_elev_MEAN,wtd_elev_STD,wtd_elev_SUM,wtd_elev_VARIETY,wtd_elev_MAJORITY,wtd_elev_MINORITY,wtd_elev_MEDIAN
cat_ID_txt,,,,,,,,,,,,
75004200007057.0,258675.00,6466875.00,0,1443,1443,430.06,372.54,111247003.00,1444,5,1422,343
75004300006312.0,6167928.00,154198200.00,59,569,510,250.99,117.77,1548112719.00,511,126,569,227
75004300001906.0,3674802.00,91870050.00,3,767,764,401.48,119.92,1475371075.00,765,377,3,402
75004300000100.0,1158019.00,28950475.00,34,415,381,156.26,85.69,180956824.00,382,91,415,128
75004300004983.0,609413.00,15235325.00,21,1468,1447,672.58,333.96,409876549.00,1448,336,1467,651
...,...,...,...,...,...,...,...,...,...,...,...,...
43933.0,328912.00,32891200.00,0,3324,3324,659.88,420.19,217042673.00,2275,200,1355,654
43973.0,172326.00,17232600.00,1,3324,3323,487.63,501.77,84030647.00,2021,254,1164,394
44553.0,508260.00,50826000.00,0,3324,3324,617.58,428.77,313889063.00,2515,45,1493,592


In [39]:
# Make watershed slope df
wtd_sl_df = pd.DataFrame()
wtd_sl_field_list = []
for field in arcpy.ListFields(wtd_slope):
    wtd_sl_field_list.append(field.name)
wtd_sl_arr = arcpy.da.TableToNumPyArray(wtd_slope, wtd_sl_field_list)
wtd_sl_df = pd.DataFrame(wtd_sl_arr)
wtd_sl_df = wtd_sl_df.drop(["OBJECTID", "wtd_slope_ZONE_CODE"],axis=1)
wtd_sl_df = wtd_sl_df.set_index('cat_ID_txt')
dfs.append(wtd_sl_df)
wtd_sl_df

,wtd_slope_COUNT,wtd_slope_AREA,wtd_slope_MIN,wtd_slope_MAX,wtd_slope_RANGE,wtd_slope_MEAN,wtd_slope_STD,wtd_slope_SUM,wtd_slope_MEDIAN,wtd_slope_PCT90,region,cat_ID_con
cat_ID_txt,,,,,,,,,,,,
75004200007057.0,258675.00,6466875.00,0.00,68.24,68.24,22.42,13.44,5800405.06,22.49,40.33,Cook_Inlet,Cook_Inlet_75004200007057.0
75004300006312.0,6167928.00,154198200.00,0.00,46.85,46.85,5.60,4.54,34538223.06,5.11,11.48,Cook_Inlet,Cook_Inlet_75004300006312.0
75004300001906.0,3674802.00,91870050.00,0.00,67.73,67.73,8.74,10.37,32127520.68,5.30,26.62,Cook_Inlet,Cook_Inlet_75004300001906.0
75004300000100.0,1158019.00,28950475.00,0.00,51.12,51.12,4.20,3.95,4860942.85,3.93,8.80,Cook_Inlet,Cook_Inlet_75004300000100.0
75004300004983.0,609413.00,15235325.00,0.00,67.44,67.44,20.99,12.28,12793041.78,19.11,38.72,Cook_Inlet,Cook_Inlet_75004300004983.0
...,...,...,...,...,...,...,...,...,...,...,...,...
43933.0,328912.00,32891200.00,0.00,2.49,2.49,0.37,0.20,123130.07,0.35,0.63,Prince_William_Sound,Prince_William_Sound_43933.0
43973.0,172326.00,17232600.00,0.00,1.51,1.51,0.34,0.18,58079.41,0.33,0.56,Prince_William_Sound,Prince_William_Sound_43973.0
44553.0,508260.00,50826000.00,0.00,2.01,2.01,0.28,0.19,144425.55,0.26,0.53,Prince_William_Sound,Prince_William_Sound_44553.0


In [40]:
# Make watershed north df
wtd_n_df = pd.DataFrame()
wtd_n_field_list = []
for field in arcpy.ListFields(wtd_per_north):
    wtd_n_field_list.append(field.name)
wtd_n_arr = arcpy.da.TableToNumPyArray(wtd_per_north,wtd_n_field_list)
wtd_n_df = pd.DataFrame(wtd_n_arr)
wtd_n_df = wtd_n_df.drop("OBJECTID",axis=1)
wtd_n_df = wtd_n_df.set_index('cat_ID_txt')
dfs.append(wtd_n_df)
wtd_n_df

,non_north_area,north_area,region,wtd_north_per,cat_ID_con
cat_ID_txt,,,,,
75004200007057.0,5612475.00,854400.00,Cook_Inlet,13.21,Cook_Inlet_75004200007057.0
75004300006312.0,99444100.00,54754100.00,Cook_Inlet,35.51,Cook_Inlet_75004300006312.0
75004300001906.0,77036175.00,14833875.00,Cook_Inlet,16.15,Cook_Inlet_75004300001906.0
75004300000100.0,17989525.00,10960950.00,Cook_Inlet,37.86,Cook_Inlet_75004300000100.0
75004300004983.0,10671675.00,4563650.00,Cook_Inlet,29.95,Cook_Inlet_75004300004983.0
...,...,...,...,...,...
43933.0,28684300.00,4206900.00,Prince_William_Sound,12.79,Prince_William_Sound_43933.0
43973.0,13954700.00,3277900.00,Prince_William_Sound,19.02,Prince_William_Sound_43973.0
44553.0,42757600.00,8068400.00,Prince_William_Sound,15.87,Prince_William_Sound_44553.0


In [41]:
# Make watershed wetland df
wtd_wet_df = pd.DataFrame()
wtd_wet_field_list = []
for field in arcpy.ListFields(wtd_wet):
    wtd_wet_field_list.append(field.name)
wtd_wet_arr = arcpy.da.TableToNumPyArray(wtd_wet,wtd_wet_field_list)
wtd_wet_df = pd.DataFrame(wtd_wet_arr)
wtd_wet_df = wtd_wet_df.drop("OBJECTID",axis=1)
wtd_wet_df = wtd_wet_df.set_index('cat_ID_txt')
dfs.append(wtd_wet_df)
wtd_wet_df

,non_wetland_area,wetland_area,region,wtd_wet_per,cat_ID_con
cat_ID_txt,,,,,
75004200007057.0,6461475.00,5400.00,Cook_Inlet,0.08,Cook_Inlet_75004200007057.0
75004300006312.0,130671950.00,23526250.00,Cook_Inlet,15.26,Cook_Inlet_75004300006312.0
75004300001906.0,73786825.00,18083225.00,Cook_Inlet,19.68,Cook_Inlet_75004300001906.0
75004300000100.0,18462700.00,10487775.00,Cook_Inlet,36.23,Cook_Inlet_75004300000100.0
75004300004983.0,15124625.00,110700.00,Cook_Inlet,0.73,Cook_Inlet_75004300004983.0
...,...,...,...,...,...
43933.0,32833700.00,57500.00,Prince_William_Sound,0.17,Prince_William_Sound_43933.0
43973.0,17164500.00,68100.00,Prince_William_Sound,0.40,Prince_William_Sound_43973.0
44553.0,50420300.00,405700.00,Prince_William_Sound,0.80,Prince_William_Sound_44553.0


In [42]:
# Make watershed lakes df
wtd_lp_df = pd.DataFrame()
wtd_lp_field_list = []
for field in arcpy.ListFields(wtd_lp):
    wtd_lp_field_list.append(field.name)
wtd_lp_arr = arcpy.da.TableToNumPyArray(wtd_lp, wtd_lp_field_list)
wtd_lp_df = pd.DataFrame(wtd_lp_arr)
wtd_lp_df = wtd_lp_df.drop("OBJECTID",axis=1)
wtd_lp_df = wtd_lp_df.set_index('cat_ID_txt')
dfs.append(wtd_lp_df)
wtd_lp_df

,FType,wtd_lake_area_sqm,wtd_lake_per,region,cat_ID,cat_ID_con
cat_ID_txt,,,,,,
75000100000004.0,390,922030.93,0.54,Cook_Inlet,75000100000004.0,Cook_Inlet_75000100000004.0
75000100000059.0,390,314031.76,0.87,Cook_Inlet,75000100000059.0,Cook_Inlet_75000100000059.0
75000100000652.0,390,32224328.96,9.98,Cook_Inlet,75000100000652.0,Cook_Inlet_75000100000652.0
75000100000981.0,390,753288.15,0.69,Cook_Inlet,75000100000981.0,Cook_Inlet_75000100000981.0
75000100001919.0,390,391083.09,0.54,Cook_Inlet,75000100001919.0,Cook_Inlet_75000100001919.0
...,...,...,...,...,...,...
43185.0,390,51132.56,0.21,Prince_William_Sound,43185.0,Prince_William_Sound_43185.0
43933.0,390,484188.75,1.47,Prince_William_Sound,43933.0,Prince_William_Sound_43933.0
43973.0,390,12804.67,0.07,Prince_William_Sound,43973.0,Prince_William_Sound_43973.0


In [43]:
# Make watershed glacier df
wtd_glac_df = pd.DataFrame()
wtd_glac_field_list = []
for field in arcpy.ListFields(wtd_glac):
    wtd_glac_field_list.append(field.name)
wtd_glac_arr = arcpy.da.TableToNumPyArray(wtd_glac, wtd_glac_field_list)
wtd_glac_df = pd.DataFrame(wtd_glac_arr)
wtd_glac_df = wtd_glac_df.drop("OBJECTID",axis=1)
wtd_glac_df = wtd_glac_df.set_index('cat_ID_txt')
dfs.append(wtd_glac_df)
wtd_glac_df


,O1Region,wtd_glacier_area_sqm,wtd_glacier_per,region,cat_ID,cat_ID_con
cat_ID_txt,,,,,,
75000100002463.0,1,170415.67,0.05,Cook_Inlet,75000100002463.0,Cook_Inlet_75000100002463.0
75000100003094.0,1,170853.46,0.07,Cook_Inlet,75000100003094.0,Cook_Inlet_75000100003094.0
75000200000021.0,1,1730403.02,0.22,Cook_Inlet,75000200000021.0,Cook_Inlet_75000200000021.0
75000200000745.0,1,2245823873.64,7.02,Cook_Inlet,75000200000745.0,Cook_Inlet_75000200000745.0
75000200002162.0,1,1728702.43,1.60,Cook_Inlet,75000200002162.0,Cook_Inlet_75000200002162.0
...,...,...,...,...,...,...
43933.0,1,1452068.36,4.41,Prince_William_Sound,43933.0,Prince_William_Sound_43933.0
43973.0,1,533700.00,3.10,Prince_William_Sound,43973.0,Prince_William_Sound_43973.0
44553.0,1,24056095.14,47.33,Prince_William_Sound,44553.0,Prince_William_Sound_44553.0


In [50]:
# Merge all data frames together
from functools import reduce
df_final = reduce(lambda left,right: pd.merge(left,right,on='cat_ID_txt',how="outer"), dfs)
#df_final = pd.concat(dfs)
df_final


,cat_elev_COUNT,cat_elev_AREA,cat_elev_MIN,cat_elev_MAX,cat_elev_RANGE,cat_elev_MEAN,cat_elev_STD,cat_elev_SUM,cat_elev_VARIETY,cat_elev_MAJORITY,...,wtd_lake_per,region_x,cat_ID_x,cat_ID_con_x,O1Region,wtd_glacier_area_sqm,wtd_glacier_per,region_y,cat_ID_y,cat_ID_con_y
cat_ID_txt,,,,,,,,,,,,,,,,,,,,,
75004200007057.0,38470.00,961750.00,0.00,908.00,908.00,346.00,251.07,13310762.00,909.00,4.00,...,0.68,Cook_Inlet,75004200007057.0,Cook_Inlet_75004200007057.0,1,245412.94,3.79,Cook_Inlet,75004200007057.0,Cook_Inlet_75004200007057.0
75004300006312.0,57563.00,1439075.00,59.00,98.00,39.00,76.31,9.37,4392462.00,40.00,69.00,...,0.02,Cook_Inlet,75004300006312.0,Cook_Inlet_75004300006312.0,NaN,NaN,NaN,NaN,NaN,NaN
75004300001906.0,12127.00,303175.00,3.00,146.00,143.00,27.85,21.37,337746.00,142.00,9.00,...,3.43,Cook_Inlet,75004300001906.0,Cook_Inlet_75004300001906.0,NaN,NaN,NaN,NaN,NaN,NaN
75004300000100.0,78759.00,1968975.00,33.00,78.00,45.00,58.67,8.23,4620998.00,46.00,65.00,...,0.13,Cook_Inlet,75004300000100.0,Cook_Inlet_75004300000100.0,NaN,NaN,NaN,NaN,NaN,NaN
75004300004983.0,113568.00,2839200.00,21.00,479.00,458.00,265.10,100.44,30107239.00,459.00,336.00,...,0.95,Cook_Inlet,75004300004983.0,Cook_Inlet_75004300004983.0,1,448126.38,2.94,Cook_Inlet,75004300004983.0,Cook_Inlet_75004300004983.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43933.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.47,Prince_William_Sound,43933.0,Prince_William_Sound_43933.0,1,1452068.36,4.41,Prince_William_Sound,43933.0,Prince_William_Sound_43933.0
43973.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.07,Prince_William_Sound,43973.0,Prince_William_Sound_43973.0,1,533700.00,3.10,Prince_William_Sound,43973.0,Prince_William_Sound_43973.0
44553.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,Prince_William_Sound,44553.0,Prince_William_Sound_44553.0,1,24056095.14,47.33,Prince_William_Sound,44553.0,Prince_William_Sound_44553.0
